# とりあえず精度0.6をぎり超えるmodelを共有します。

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5) # 畳み込み層(入力ch,フィルタ,そのサイズ)
        self.pool = nn.MaxPool2d(2, 2) # プーリング層(領域サイズ,ストライド)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5) # 2つめの畳み込み層
        self.fc1 = nn.Linear(16 * 5 * 5, 256) # 全結合層
        self.dropout = nn.Dropout(p=0.5) # ドロップアウト(ドロップアウト率)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)
        self.bn1 = nn.BatchNorm2d(6)
        self.bn2 = nn.BatchNorm1d(256)
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        return x
model = DNN().to(device)
model

- 修正箇所1 -->  train_dataloaderはimage と labelをセットで返すため、二つを受け取るようにする。
- 修正箇所2 --> modelの入力データの形に合わせる(ここでは列数にfc1の入力28*28を指定)
- 修正箇所3 --> loss_sum は GPUで計算した値であり、GPU上のメモリに存在する。また勾配情報はここでは必要ではないため.item()とする。
- 修正箇所4, 修正箇所5 --> 修正箇所1と同様の理由によりリストとして受け取り、imageとlabelの二つを分ける。
- 修正箇所6 --> 修正箇所2 と同様